In [43]:
import facebook
import requests
import json 

In [44]:
def get_page_credentials(company_name, page_name):
    with open("credentials.json", "r") as f:
        data = json.load(f)

    for company in data["companies"]:
        if company["company_name"] == company_name:
            for page in company["pages"]:
                if page["page_name"] == page_name:
                    return page["page_id"], page["access_token"]
            break
    return None, None

In [45]:
page_id, access_token = get_page_credentials("raisebe","Facebook")
print(page_id,access_token)

107246428673184 EAAI569D4SBkBO1pg7zkAM2nqOwl3jnzI1bo2muZAkvBO7e2aMKWzpU5VBHP6POByn7H2woNf0YqLEqnkpTlL2CHkYBe1ypCsZCAhvmss7ZA68hTbQ1cgdagFGlUVZCuZCWM1WZAzCvNOORD3bqi8TI6UnUCil6pGDmp84h4CanpxgCEEujhAMaveAn0pKbMYJc8DQuqmohykRZCW5ZAaffmx81EZD


In [46]:
def post_text_status(company_name,page_name, status_message):
    page_id, access_token = get_page_credentials(company_name,page_name)
    
    if page_id and access_token:
        print(page_id,access_token)
        graph = facebook.GraphAPI(access_token=access_token, version="3.0")
        graph.put_object(parent_object=page_id, connection_name='feed', message=status_message)
        posts = graph.get_connections(id=page_id, connection_name="posts", limit=1)
        print("Text status posted successfully.")
        print(posts['data'][0]['id'])
        return posts['data'][0]['id']
    else:
        print("Page not found or credentials not available.")

In [47]:
def post_photo(company_name ,page_name, photo_path, caption=None):
    page_id, access_token = get_page_credentials(company_name ,page_name)

    if page_id and access_token:
        graph = facebook.GraphAPI(access_token=access_token, version="3.0")
        with open(photo_path, "rb") as photo:
            graph.put_photo(parent_object=page_id, image=photo, message=caption)
            posts = graph.get_connections(id=page_id, connection_name="posts", limit=1)
            print("Photo posted successfully.")
            print(posts['data'][0]['id'])
            return posts['data'][0]['id']
    else:
        print("Page not found or credentials not available.")

In [48]:
def post_link(company_name ,page_name, link_url, link_caption, link_description):
    page_id, access_token = get_page_credentials(company_name ,page_name)
    
    if not page_id or not access_token:
        print("Unable to retrieve credentials. Cannot post link.")
        return
    
    graph = facebook.GraphAPI(access_token=access_token, version="3.0")
    
    message = f"{link_caption}\n{link_description}"

    try:
        graph.put_object(parent_object=page_id, connection_name='feed', link=link_url, message=message)
        posts = graph.get_connections(id=page_id, connection_name="posts", limit=1)
        print("Link posted successfully.")
        print(posts['data'][0]['id'])
        return posts['data'][0]['id']
    except facebook.GraphAPIError as e:
        print(f"Error posting link: {e}")
        return None

In [49]:
import requests
from tqdm import tqdm  # Import tqdm library for the progress bar

def post_video(company_name , page_name, video_path, video_title, video_description):
    page_id, access_token = get_page_credentials(company_name ,page_name)
    
    if not page_id or not access_token:
        print("Unable to retrieve credentials. Cannot post video.")
        return
    
    url = f'https://graph.facebook.com/{page_id}/videos'
    
    params = {
        'access_token': access_token,
        'description': video_description
    }

    with open(video_path, 'rb') as video_file:
        files = {
            'source': (video_title, video_file, 'video/mp4')
        }

        response = requests.post(url, data=params, files=files, stream=True)  # Use stream=True for streaming upload

        if response.status_code == 200:
            print("Video posted successfully.")
            return response.json()
        else:
            print(f"Error posting video: {response.status_code}")
            print(response.content)  # Print detailed response content for debugging
            return None




In [50]:
def create_event(company_name , page_name, event_name, start_time, end_time, event_description):
    page_id, access_token = get_page_credentials(company_name ,page_name)

    if page_id and access_token:
        graph = facebook.GraphAPI(access_token=access_token, version="3.0")
        event_data = {
            "name": event_name,
            "start_time": start_time,
            "end_time": end_time,
            "description": event_description,
        }
        graph.put_object(parent_object=page_id, connection_name="events", **event_data)
        posts = graph.get_connections(id=page_id, connection_name="posts", limit=1)
        print("Event created successfully.")
        print(posts['data'][0]['id'])
        return posts['data'][0]['id']
    else:
        print("Page not found or credentials not available.")

In [51]:
def delete_post(company_name ,page_name, post_id):
    page_id, access_token = get_page_credentials(company_name ,page_name)

    if page_id and access_token:
        graph = facebook.GraphAPI(access_token=access_token, version="3.0")
        graph.delete_object(id=post_id)
        print("Post deleted successfully.")
    else:
        print("Page not found or credentials not available.")

In [52]:
def get_page_info(company_name, page_name):
    page_id, access_token = get_page_credentials(company_name, page_name)

    if page_id and access_token:
        graph = facebook.GraphAPI(access_token=access_token, version="3.0")
        page_info = graph.get_object(id=page_id, fields="name, category, description, picture, cover")
        return page_info
    else:
        return None

In [53]:
def get_page_posts(company_name , page_name, post_count=5):
    page_id, access_token = get_page_credentials(company_name ,page_name)

    if page_id and access_token:
        graph = facebook.GraphAPI(access_token=access_token, version="3.0")
        posts = graph.get_connections(id=page_id, connection_name="posts", limit=post_count)
        return posts
    else:
        return None

In [54]:

get_page_posts("raisebe" , "Facebook", post_count=5)

{'data': [{'created_time': '2023-08-25T11:36:45+0000',
   'message': 'Check out this cool website!\nExplore amazing content and more.',
   'id': '107246428673184_273103388812657'},
  {'created_time': '2023-08-25T11:36:38+0000',
   'message': 'Peakpx.jpg',
   'id': '107246428673184_273103358812660'},
  {'created_time': '2023-08-25T11:36:34+0000',
   'message': 'Hello, World!',
   'id': '107246428673184_273103295479333'},
  {'created_time': '2023-08-25T11:34:11+0000',
   'message': 'Check out this cool website!\nExplore amazing content and more.',
   'id': '107246428673184_273102382146091'},
  {'created_time': '2023-08-25T11:34:04+0000',
   'message': 'Peakpx.jpg',
   'id': '107246428673184_273102328812763'}],
 'paging': {'cursors': {'before': 'QVFIUnlQNTEyZAVQ1bk9HYTY4YWdzNGw5UFFsWTBKTDlWR0hEczAweUdnS3NzNERva2tTcnM0OEtBWEE5blNCZAGRDVFJjTEZALOE52NTkyVlF6R1ZAxb3BORVNBUGxiSmppM0I2djl4LU02Rk1YTnJQWnFhVVFRN1dfRWkzQ3VBLUM0bUQ2UVFoX0s0T2VueWlsd3lGN21hd2JtS1VoUFotVkI1bTI0NXhFbDdxRlNQWXZApLVNRQ2

In [55]:
def get_page_insights(company_name, page_name, metric, period="day"):
    page_id, access_token = get_page_credentials(company_name ,page_name)

    if page_id and access_token:
        graph = facebook.GraphAPI(access_token=access_token, version="3.0")
        insights = graph.get_object(id=page_id, fields=f"insights.metric({metric}).period({period})")
        return insights.get("data", [])
    else:
        return None

In [59]:
import pandas as pd
from datetime import datetime

def runallpost():
    df = pd.read_excel("posts.xlsx")
    
    for index, row in df.iterrows():
        company_name = row['Company Name']
        page_name = row['Page Name']
        post_type = row['Post Type']
        post_status = row['Posted']


        page_id, access_token = get_page_credentials(company_name, page_name)

        
        if post_status != "Posted":
            if not (page_id and access_token):
                print(f"Credentials not found for {company_name} - {page_name}. Skipping post.")
                continue
            

            if post_type == 'Text':
                status_message = row['Status Message']
                post_id = post_text_status(company_name, page_name, status_message)
            elif post_type == 'Photo':
                photo_url = row['Photo URL']
                caption = row['Caption']
                post_id = post_photo(company_name, page_name, photo_url, caption)
            elif post_type == 'Link':
                link_url = row['Link URL']
                link_caption = row['Link Caption']
                link_description = row['Link Description']
                post_id = post_link(company_name, page_name, link_url, link_caption, link_description)
            elif post_type == 'Video':
                print("Function is under construction for Event posts.")
                # Similar to other post types, call the relevant function and get the post_id
                # post_id = post_video(company_name, page_name, video_url, title, description)
                post_id =''
            elif post_type == 'Event':
                print("Function is under construction for Event posts.")
                # Similar to other post types, call the relevant function and get the post_id
                # post_id = create_event(company_name, page_name, event_name, start_time, end_time, event_location, description)
                post_id =''
            else:
                print(f"Unknown post type '{post_type}' for {company_name} - {page_name}. Skipping post.")
                continue
            
            # Record the post_id and status in the DataFrame
            df.at[index, 'Post_ID'] = post_id
            print(post_id)
            df.at[index, 'Posted'] = 'Posted' if post_id else 'Failed'
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            df.at[index, 'created_time'] = current_time

    df.to_excel("posts.xlsx", index=False)



In [60]:
runallpost()

107246428673184 EAAI569D4SBkBO1pg7zkAM2nqOwl3jnzI1bo2muZAkvBO7e2aMKWzpU5VBHP6POByn7H2woNf0YqLEqnkpTlL2CHkYBe1ypCsZCAhvmss7ZA68hTbQ1cgdagFGlUVZCuZCWM1WZAzCvNOORD3bqi8TI6UnUCil6pGDmp84h4CanpxgCEEujhAMaveAn0pKbMYJc8DQuqmohykRZCW5ZAaffmx81EZD
Text status posted successfully.
107246428673184_273110028811993
107246428673184_273110028811993
Photo posted successfully.
107246428673184_273110092145320
107246428673184_273110092145320
Link posted successfully.
107246428673184_273110142145315
107246428673184_273110142145315
Function is under construction for Event posts.

Function is under construction for Event posts.

Unknown post type 'media' for raisebe - Instagram. Skipping post.
Unknown post type 'media_link' for raisebe - Instagram. Skipping post.
Unknown post type 'media' for raisebe - Instagram. Skipping post.
Unknown post type 'media_link' for raisebe - Instagram. Skipping post.
